# HTS Coil Performance Comparison: Baseline vs High-Field Designs

This notebook compares baseline (2.1T) and high-field (7.07T) HTS coil designs as presented in the accompanying research paper. We analyze scaling relationships, performance trade-offs, and engineering feasibility to understand the implications of pursuing higher magnetic field strengths in REBCO superconducting systems.

## Notebook Objectives

1. **Compare Design Parameters**: Analyze differences between baseline and high-field configurations
2. **Evaluate Performance Metrics**: Field strength, ripple performance, thermal margins, and material costs
3. **Assess Engineering Feasibility**: Manufacturing challenges, operational considerations, and practical limitations
4. **Quantify Scaling Relationships**: How performance metrics scale with field strength requirements
5. **Cost-Benefit Analysis**: Economic implications of high-field designs

## Design Configurations Under Study

**Baseline Configuration:**
- Target Field: 2.1 T
- Field Ripple: 0.01%
- Number of Turns: 400
- Operating Current: 1,171 A

**High-Field Configuration:**
- Target Field: 7.07 T
- Field Ripple: 0.16%
- Number of Turns: 1,000
- Operating Current: 1,800 A

---

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import constants
import warnings
warnings.filterwarnings('ignore')

# Set up plotting parameters
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['grid.alpha'] = 0.3

print("Libraries imported successfully")
print("Ready for HTS coil design comparison analysis")

## 1. Design Parameter Comparison

Let's start by defining both configurations and examining the fundamental differences in their design parameters.

In [ ]:
# Define baseline and high-field configurations
baseline_config = {
    'name': 'Baseline Design',
    'field_target': 2.1,  # Tesla
    'field_ripple': 0.01,  # Percent
    'turns': 400,
    'current': 1171,  # Amperes
    'inner_radius': 1.0,  # meters (estimated)
    'outer_radius': 1.5,  # meters (estimated)
    'height': 2.0,  # meters (estimated)
    'layers': 20,  # estimated
    'temperature': 20,  # Kelvin (LN2 cooling)
    'tape_width': 12e-3,  # meters (12mm REBCO tape)
    'tape_thickness': 100e-6  # meters (100 micron)
}

high_field_config = {
    'name': 'High-Field Design',
    'field_target': 7.07,  # Tesla
    'field_ripple': 0.16,  # Percent
    'turns': 1000,
    'current': 1800,  # Amperes
    'inner_radius': 0.8,  # meters (estimated, more compact)
    'outer_radius': 1.8,  # meters (estimated, thicker build)
    'height': 2.5,  # meters (estimated, taller)
    'layers': 50,  # estimated, more layers for higher field
    'temperature': 4.2,  # Kelvin (liquid helium cooling)
    'tape_width': 12e-3,  # meters (12mm REBCO tape)
    'tape_thickness': 100e-6  # meters (100 micron)
}

configs = [baseline_config, high_field_config]

# Create comparison table
comparison_data = {
    'Parameter': ['Field Target (T)', 'Field Ripple (%)', 'Number of Turns', 'Current (A)', 
                 'Inner Radius (m)', 'Outer Radius (m)', 'Height (m)', 'Layers', 
                 'Operating Temperature (K)', 'Tape Width (mm)', 'Tape Thickness (μm)'],
    'Baseline Design': [baseline_config['field_target'], baseline_config['field_ripple'], 
                       baseline_config['turns'], baseline_config['current'],
                       baseline_config['inner_radius'], baseline_config['outer_radius'],
                       baseline_config['height'], baseline_config['layers'],
                       baseline_config['temperature'], baseline_config['tape_width']*1000,
                       baseline_config['tape_thickness']*1e6],
    'High-Field Design': [high_field_config['field_target'], high_field_config['field_ripple'],
                         high_field_config['turns'], high_field_config['current'],
                         high_field_config['inner_radius'], high_field_config['outer_radius'],
                         high_field_config['height'], high_field_config['layers'],
                         high_field_config['temperature'], high_field_config['tape_width']*1000,
                         high_field_config['tape_thickness']*1e6]
}

df_comparison = pd.DataFrame(comparison_data)
df_comparison['Ratio (High/Base)'] = df_comparison['High-Field Design'] / df_comparison['Baseline Design']

print("Design Parameter Comparison:")
print("=" * 60)
print(df_comparison.to_string(index=False, float_format='%.3f'))

# Calculate key scaling factors
field_ratio = high_field_config['field_target'] / baseline_config['field_target']
current_ratio = high_field_config['current'] / baseline_config['current']
turns_ratio = high_field_config['turns'] / baseline_config['turns']

print(f"\nKey Scaling Factors:")
print(f"Field Strength Ratio: {field_ratio:.2f}x")
print(f"Current Ratio: {current_ratio:.2f}x")
print(f"Turns Ratio: {turns_ratio:.2f}x")
print(f"Ampere-Turns Ratio: {current_ratio * turns_ratio:.2f}x")

## 2. Performance Metrics Analysis

Now let's analyze key performance metrics and understand how they scale between the two designs.

In [ ]:
# Calculate additional performance metrics for both configurations
def calculate_performance_metrics(config):
    """Calculate comprehensive performance metrics for a coil configuration"""
    
    # Geometric calculations
    coil_volume = np.pi * (config['outer_radius']**2 - config['inner_radius']**2) * config['height']
    radial_build = config['outer_radius'] - config['inner_radius']
    aspect_ratio = config['height'] / radial_build
    
    # Electromagnetic calculations
    ampere_turns = config['current'] * config['turns']
    current_density = config['current'] / (config['tape_width'] * config['tape_thickness'])
    stored_energy = 0.5 * 4 * np.pi * 1e-7 * (ampere_turns**2) * coil_volume / (config['outer_radius'] - config['inner_radius'])  # Approximate
    
    # Material calculations
    avg_radius = (config['inner_radius'] + config['outer_radius']) / 2
    tape_length = config['turns'] * 2 * np.pi * avg_radius
    tape_volume = tape_length * config['tape_width'] * config['tape_thickness']
    
    # REBCO tape cost (approximate: $200/meter for 12mm wide tape)
    tape_cost_per_meter = 200  # USD
    total_tape_cost = tape_length * tape_cost_per_meter / 1e6  # Million USD
    
    # Thermal calculations
    # Approximate AC losses (simplified)
    ac_losses = 1e-6 * (config['current']**2) * tape_length  # Very rough estimate in Watts
    
    # Cooling power requirements (rough estimates)
    if config['temperature'] < 10:  # Helium cooling
        cooling_efficiency = 0.01  # 1% Carnot efficiency for 4K cooling
    else:  # Nitrogen cooling
        cooling_efficiency = 0.1   # 10% efficiency for 20K cooling
    
    cooling_power = ac_losses / cooling_efficiency  # Watts
    
    # Mechanical stress (simplified hoop stress)
    magnetic_pressure = (config['field_target']**2) / (2 * constants.mu_0) * 1e-6  # MPa
    hoop_stress = magnetic_pressure * config['inner_radius'] / radial_build  # Simplified
    
    return {
        'coil_volume': coil_volume,
        'radial_build': radial_build,
        'aspect_ratio': aspect_ratio,
        'ampere_turns': ampere_turns,
        'current_density': current_density / 1e6,  # MA/m²
        'stored_energy': stored_energy / 1e6,  # MJ
        'tape_length': tape_length / 1000,  # km
        'tape_volume': tape_volume,
        'tape_cost': total_tape_cost,
        'ac_losses': ac_losses / 1000,  # kW
        'cooling_power': cooling_power / 1000,  # kW
        'magnetic_pressure': magnetic_pressure,
        'hoop_stress': hoop_stress
    }

# Calculate metrics for both configurations
baseline_metrics = calculate_performance_metrics(baseline_config)
high_field_metrics = calculate_performance_metrics(high_field_config)

# Create performance comparison table
performance_data = {
    'Metric': ['Coil Volume (m³)', 'Radial Build (m)', 'Aspect Ratio', 'Ampere-Turns (MA)', 
              'Current Density (MA/m²)', 'Stored Energy (MJ)', 'Tape Length (km)', 
              'Tape Cost ($M)', 'AC Losses (kW)', 'Cooling Power (kW)', 
              'Magnetic Pressure (MPa)', 'Hoop Stress (MPa)'],
    'Baseline': [baseline_metrics['coil_volume'], baseline_metrics['radial_build'], 
                baseline_metrics['aspect_ratio'], baseline_metrics['ampere_turns']/1e6,
                baseline_metrics['current_density'], baseline_metrics['stored_energy'],
                baseline_metrics['tape_length'], baseline_metrics['tape_cost'],
                baseline_metrics['ac_losses'], baseline_metrics['cooling_power'],
                baseline_metrics['magnetic_pressure'], baseline_metrics['hoop_stress']],
    'High-Field': [high_field_metrics['coil_volume'], high_field_metrics['radial_build'],
                  high_field_metrics['aspect_ratio'], high_field_metrics['ampere_turns']/1e6,
                  high_field_metrics['current_density'], high_field_metrics['stored_energy'],
                  high_field_metrics['tape_length'], high_field_metrics['tape_cost'],
                  high_field_metrics['ac_losses'], high_field_metrics['cooling_power'],
                  high_field_metrics['magnetic_pressure'], high_field_metrics['hoop_stress']]
}

df_performance = pd.DataFrame(performance_data)
df_performance['Scaling Factor'] = df_performance['High-Field'] / df_performance['Baseline']

print("\nPerformance Metrics Comparison:")
print("=" * 70)
print(df_performance.to_string(index=False, float_format='%.3f'))

## 3. Scaling Relationships Visualization

Let's visualize how key performance metrics scale with magnetic field strength.

In [ ]:
# Create comprehensive visualization of scaling relationships
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Data for plotting
designs = ['Baseline\n(2.1 T)', 'High-Field\n(7.07 T)']
fields = [baseline_config['field_target'], high_field_config['field_target']]
colors = ['#1f77b4', '#ff7f0e']

# Plot 1: Current and Turns comparison
ax1 = axes[0, 0]
currents = [baseline_config['current'], high_field_config['current']]
turns = [baseline_config['turns'], high_field_config['turns']]

x = np.arange(len(designs))
width = 0.35

bars1 = ax1.bar(x - width/2, currents, width, label='Current (A)', color=colors[0], alpha=0.7)
bars2 = ax1.bar(x + width/2, turns, width, label='Turns', color=colors[1], alpha=0.7)

ax1.set_xlabel('Design Configuration')
ax1.set_ylabel('Value')
ax1.set_title('Current and Turns Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(designs)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add value labels
for bar, value in zip(bars1, currents):
    height = bar.get_height()
    ax1.annotate(f'{value}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
for bar, value in zip(bars2, turns):
    height = bar.get_height()
    ax1.annotate(f'{value}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

# Plot 2: Field Strength vs Ripple Trade-off
ax2 = axes[0, 1]
field_strengths = [baseline_config['field_target'], high_field_config['field_target']]
ripples = [baseline_config['field_ripple'], high_field_config['field_ripple']]

scatter = ax2.scatter(field_strengths, ripples, s=200, c=colors, alpha=0.7, edgecolors='black')
ax2.set_xlabel('Field Strength (T)')
ax2.set_ylabel('Field Ripple (%)')
ax2.set_title('Field Strength vs Ripple Trade-off')
ax2.grid(True, alpha=0.3)

# Add labels
for i, design in enumerate(['Baseline', 'High-Field']):
    ax2.annotate(design, (field_strengths[i], ripples[i]), 
                xytext=(10, 10), textcoords='offset points')

# Plot 3: Cost Scaling
ax3 = axes[0, 2]
costs = [baseline_metrics['tape_cost'], high_field_metrics['tape_cost']]
stored_energies = [baseline_metrics['stored_energy'], high_field_metrics['stored_energy']]

bars3 = ax3.bar(designs, costs, color=colors, alpha=0.7)
ax3.set_ylabel('Tape Cost (Million $)')
ax3.set_title('Material Cost Comparison')
ax3.grid(True, alpha=0.3)

for bar, value in zip(bars3, costs):
    height = bar.get_height()
    ax3.annotate(f'${value:.1f}M', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

# Plot 4: Power Requirements
ax4 = axes[1, 0]
ac_losses = [baseline_metrics['ac_losses'], high_field_metrics['ac_losses']]
cooling_powers = [baseline_metrics['cooling_power'], high_field_metrics['cooling_power']]

bars4a = ax4.bar(x - width/2, ac_losses, width, label='AC Losses (kW)', color='red', alpha=0.7)
bars4b = ax4.bar(x + width/2, cooling_powers, width, label='Cooling Power (kW)', color='blue', alpha=0.7)

ax4.set_xlabel('Design Configuration')
ax4.set_ylabel('Power (kW)')
ax4.set_title('Power Requirements Comparison')
ax4.set_xticks(x)
ax4.set_xticklabels(designs)
ax4.legend()
ax4.grid(True, alpha=0.3)

# Plot 5: Mechanical Stress
ax5 = axes[1, 1]
pressures = [baseline_metrics['magnetic_pressure'], high_field_metrics['magnetic_pressure']]
stresses = [baseline_metrics['hoop_stress'], high_field_metrics['hoop_stress']]

bars5a = ax5.bar(x - width/2, pressures, width, label='Magnetic Pressure (MPa)', color='purple', alpha=0.7)
bars5b = ax5.bar(x + width/2, stresses, width, label='Hoop Stress (MPa)', color='orange', alpha=0.7)

ax5.set_xlabel('Design Configuration')
ax5.set_ylabel('Stress (MPa)')
ax5.set_title('Mechanical Stress Comparison')
ax5.set_xticks(x)
ax5.set_xticklabels(designs)
ax5.legend()
ax5.grid(True, alpha=0.3)

# Plot 6: Performance Efficiency
ax6 = axes[1, 2]
field_per_cost = [fields[i]/costs[i] for i in range(2)]
field_per_power = [fields[i]/cooling_powers[i] for i in range(2)]

bars6a = ax6.bar(x - width/2, field_per_cost, width, label='Field/Cost (T/$M)', color='green', alpha=0.7)
ax6_twin = ax6.twinx()
bars6b = ax6_twin.bar(x + width/2, field_per_power, width, label='Field/Power (T/kW)', color='brown', alpha=0.7)

ax6.set_xlabel('Design Configuration')
ax6.set_ylabel('Field/Cost (T/$M)', color='green')
ax6_twin.set_ylabel('Field/Power (T/kW)', color='brown')
ax6.set_title('Performance Efficiency Metrics')
ax6.set_xticks(x)
ax6.set_xticklabels(designs)
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('HTS Coil Design Comparison: Baseline vs High-Field Performance', fontsize=16, y=1.02)
plt.show()

## 4. Engineering Feasibility Assessment

Let's evaluate the engineering challenges and feasibility considerations for both designs.

In [ ]:
# Engineering feasibility analysis
def assess_feasibility(config, metrics):
    """Assess engineering feasibility based on key technical parameters"""
    
    feasibility = {}
    
    # Current density assessment
    # REBCO critical current density at different temperatures and fields
    if config['temperature'] < 10:  # Liquid helium
        jc_max = 1000  # MA/m² (very high at 4K)
    else:  # Liquid nitrogen
        jc_max = 200   # MA/m² (reduced at 20K)
    
    current_density_margin = jc_max / metrics['current_density']
    
    if current_density_margin > 3:
        cd_feasibility = 'EXCELLENT'
    elif current_density_margin > 2:
        cd_feasibility = 'GOOD'
    elif current_density_margin > 1.5:
        cd_feasibility = 'MARGINAL'
    else:
        cd_feasibility = 'CHALLENGING'
    
    feasibility['current_density'] = {
        'value': metrics['current_density'],
        'limit': jc_max,
        'margin': current_density_margin,
        'assessment': cd_feasibility
    }
    
    # Mechanical stress assessment
    # REBCO tape strength: ~100-200 MPa tensile, ~500 MPa compression
    stress_limit = 150  # MPa (conservative)
    stress_margin = stress_limit / metrics['hoop_stress']
    
    if stress_margin > 3:
        stress_feasibility = 'EXCELLENT'
    elif stress_margin > 2:
        stress_feasibility = 'GOOD'
    elif stress_margin > 1.2:
        stress_feasibility = 'MARGINAL'
    else:
        stress_feasibility = 'CHALLENGING'
    
    feasibility['mechanical_stress'] = {
        'value': metrics['hoop_stress'],
        'limit': stress_limit,
        'margin': stress_margin,
        'assessment': stress_feasibility
    }
    
    # Cooling assessment
    if config['temperature'] < 10:
        cooling_complexity = 'HIGH'  # Helium cooling is complex
        cooling_cost_factor = 10
    else:
        cooling_complexity = 'MEDIUM'  # Nitrogen cooling is simpler
        cooling_cost_factor = 1
    
    feasibility['cooling'] = {
        'temperature': config['temperature'],
        'power_required': metrics['cooling_power'],
        'complexity': cooling_complexity,
        'cost_factor': cooling_cost_factor
    }
    
    # Manufacturing complexity
    if config['layers'] > 40:
        mfg_complexity = 'HIGH'
    elif config['layers'] > 20:
        mfg_complexity = 'MEDIUM'
    else:
        mfg_complexity = 'LOW'
    
    feasibility['manufacturing'] = {
        'layers': config['layers'],
        'tape_length': metrics['tape_length'],
        'complexity': mfg_complexity
    }
    
    # Overall cost assessment
    if metrics['tape_cost'] < 10:
        cost_assessment = 'REASONABLE'
    elif metrics['tape_cost'] < 50:
        cost_assessment = 'EXPENSIVE'
    else:
        cost_assessment = 'VERY_EXPENSIVE'
    
    feasibility['cost'] = {
        'tape_cost': metrics['tape_cost'],
        'assessment': cost_assessment
    }
    
    return feasibility

# Assess feasibility for both designs
baseline_feasibility = assess_feasibility(baseline_config, baseline_metrics)
high_field_feasibility = assess_feasibility(high_field_config, high_field_metrics)

# Print feasibility assessment
def print_feasibility(name, feasibility):
    print(f"\n{name} Feasibility Assessment:")
    print("=" * 50)
    
    print(f"Current Density: {feasibility['current_density']['assessment']}")
    print(f"  Operating: {feasibility['current_density']['value']:.1f} MA/m²")
    print(f"  Limit: {feasibility['current_density']['limit']:.0f} MA/m²")
    print(f"  Margin: {feasibility['current_density']['margin']:.1f}x")
    
    print(f"\nMechanical Stress: {feasibility['mechanical_stress']['assessment']}")
    print(f"  Hoop Stress: {feasibility['mechanical_stress']['value']:.1f} MPa")
    print(f"  Limit: {feasibility['mechanical_stress']['limit']:.0f} MPa")
    print(f"  Margin: {feasibility['mechanical_stress']['margin']:.1f}x")
    
    print(f"\nCooling: {feasibility['cooling']['complexity']} Complexity")
    print(f"  Temperature: {feasibility['cooling']['temperature']:.1f} K")
    print(f"  Power Required: {feasibility['cooling']['power_required']:.1f} kW")
    
    print(f"\nManufacturing: {feasibility['manufacturing']['complexity']} Complexity")
    print(f"  Layers: {feasibility['manufacturing']['layers']}")
    print(f"  Tape Length: {feasibility['manufacturing']['tape_length']:.1f} km")
    
    print(f"\nCost: {feasibility['cost']['assessment']}")
    print(f"  Tape Cost: ${feasibility['cost']['tape_cost']:.1f} Million")

print_feasibility("Baseline Design", baseline_feasibility)
print_feasibility("High-Field Design", high_field_feasibility)

## 5. Risk Assessment and Mitigation Strategies

Let's identify key risks for each design and potential mitigation strategies.

In [ ]:
# Risk assessment framework
risk_categories = {
    'technical': ['Current density limits', 'Mechanical stress', 'Thermal management', 'Field quality'],
    'economic': ['Material costs', 'Manufacturing costs', 'Operating costs', 'Development costs'],
    'schedule': ['Development time', 'Manufacturing time', 'Testing time', 'Deployment time'],
    'operational': ['Reliability', 'Maintainability', 'Safety', 'Performance degradation']
}

# Risk scoring: 1-5 scale (1=low risk, 5=high risk)
baseline_risks = {
    'technical': {
        'current_density': 2,  # Well within limits
        'mechanical_stress': 2,  # Manageable stress levels
        'thermal_management': 2,  # Nitrogen cooling is established
        'field_quality': 1  # Excellent ripple performance
    },
    'economic': {
        'material_costs': 2,  # Moderate tape requirements
        'manufacturing_costs': 2,  # Standard complexity
        'operating_costs': 2,  # Reasonable cooling costs
        'development_costs': 2  # Lower complexity development
    },
    'schedule': {
        'development_time': 2,  # Proven technology
        'manufacturing_time': 2,  # Standard manufacturing
        'testing_time': 2,  # Routine testing
        'deployment_time': 2  # Known deployment challenges
    },
    'operational': {
        'reliability': 2,  # Good track record
        'maintainability': 2,  # Manageable maintenance
        'safety': 2,  # Standard safety protocols
        'performance_degradation': 2  # Slow degradation expected
    }
}

high_field_risks = {
    'technical': {
        'current_density': 4,  # Approaching limits
        'mechanical_stress': 4,  # High stress levels
        'thermal_management': 4,  # Complex helium cooling
        'field_quality': 3  # Higher ripple acceptable but challenging
    },
    'economic': {
        'material_costs': 4,  # High tape requirements
        'manufacturing_costs': 4,  # Complex manufacturing
        'operating_costs': 4,  # Expensive helium cooling
        'development_costs': 4  # Complex development
    },
    'schedule': {
        'development_time': 4,  # Pushing technology limits
        'manufacturing_time': 4,  # Complex manufacturing
        'testing_time': 4,  # Extensive testing required
        'deployment_time': 3  # Challenging but feasible
    },
    'operational': {
        'reliability': 3,  # Higher complexity = lower reliability
        'maintainability': 4,  # Complex maintenance
        'safety': 3,  # Higher energy, more safety concerns
        'performance_degradation': 3  # Higher stress = faster degradation
    }
}

# Create risk comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

categories = list(risk_categories.keys())
baseline_scores = [np.mean(list(baseline_risks[cat].values())) for cat in categories]
high_field_scores = [np.mean(list(high_field_risks[cat].values())) for cat in categories]

# Overall risk comparison
ax1 = axes[0, 0]
x = np.arange(len(categories))
width = 0.35

bars1 = ax1.bar(x - width/2, baseline_scores, width, label='Baseline', color='blue', alpha=0.7)
bars2 = ax1.bar(x + width/2, high_field_scores, width, label='High-Field', color='red', alpha=0.7)

ax1.set_xlabel('Risk Category')
ax1.set_ylabel('Risk Score (1-5)')
ax1.set_title('Risk Assessment by Category')
ax1.set_xticks(x)
ax1.set_xticklabels([cat.title() for cat in categories])
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 5)

# Technical risks detail
ax2 = axes[0, 1]
tech_risks = list(baseline_risks['technical'].keys())
baseline_tech = list(baseline_risks['technical'].values())
high_field_tech = list(high_field_risks['technical'].values())

x2 = np.arange(len(tech_risks))
bars3 = ax2.bar(x2 - width/2, baseline_tech, width, label='Baseline', color='blue', alpha=0.7)
bars4 = ax2.bar(x2 + width/2, high_field_tech, width, label='High-Field', color='red', alpha=0.7)

ax2.set_xlabel('Technical Risk Factor')
ax2.set_ylabel('Risk Score (1-5)')
ax2.set_title('Technical Risk Breakdown')
ax2.set_xticks(x2)
ax2.set_xticklabels([risk.replace('_', ' ').title() for risk in tech_risks], rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 5)

# Economic risks detail
ax3 = axes[1, 0]
econ_risks = list(baseline_risks['economic'].keys())
baseline_econ = list(baseline_risks['economic'].values())
high_field_econ = list(high_field_risks['economic'].values())

x3 = np.arange(len(econ_risks))
bars5 = ax3.bar(x3 - width/2, baseline_econ, width, label='Baseline', color='blue', alpha=0.7)
bars6 = ax3.bar(x3 + width/2, high_field_econ, width, label='High-Field', color='red', alpha=0.7)

ax3.set_xlabel('Economic Risk Factor')
ax3.set_ylabel('Risk Score (1-5)')
ax3.set_title('Economic Risk Breakdown')
ax3.set_xticks(x3)
ax3.set_xticklabels([risk.replace('_', ' ').title() for risk in econ_risks], rotation=45, ha='right')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 5)

# Risk-Performance matrix
ax4 = axes[1, 1]
overall_baseline_risk = np.mean([np.mean(list(baseline_risks[cat].values())) for cat in categories])
overall_high_field_risk = np.mean([np.mean(list(high_field_risks[cat].values())) for cat in categories])

performance_baseline = baseline_config['field_target']
performance_high_field = high_field_config['field_target']

ax4.scatter(overall_baseline_risk, performance_baseline, s=300, c='blue', alpha=0.7, 
           label='Baseline', edgecolors='black', linewidth=2)
ax4.scatter(overall_high_field_risk, performance_high_field, s=300, c='red', alpha=0.7, 
           label='High-Field', edgecolors='black', linewidth=2)

ax4.set_xlabel('Overall Risk Score')
ax4.set_ylabel('Field Performance (T)')
ax4.set_title('Risk vs Performance Trade-off')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Add annotations
ax4.annotate('Lower Risk\nLower Performance', xy=(overall_baseline_risk, performance_baseline),
            xytext=(0.5, 1), fontsize=10, ha='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', alpha=0.7))
ax4.annotate('Higher Risk\nHigher Performance', xy=(overall_high_field_risk, performance_high_field),
            xytext=(3.5, 6), fontsize=10, ha='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='lightcoral', alpha=0.7))

plt.tight_layout()
plt.suptitle('Comprehensive Risk Assessment: Baseline vs High-Field Designs', fontsize=16, y=1.02)
plt.show()

# Print summary
print(f"\nRisk Assessment Summary:")
print("=" * 50)
print(f"Baseline Design Overall Risk: {overall_baseline_risk:.2f}/5.0")
print(f"High-Field Design Overall Risk: {overall_high_field_risk:.2f}/5.0")
print(f"Risk Increase for High-Field: {(overall_high_field_risk/overall_baseline_risk - 1)*100:.1f}%")
print(f"Performance Increase: {(performance_high_field/performance_baseline - 1)*100:.1f}%")